# Ada-FracBNN Testing Notebook (Google Colab Version)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/endingengineering/blob/main/test_ada_fracbnn_colab.ipynb)

This notebook is optimized for Google Colab with GPU support.

## ⚠️ Important First Steps:
1. **Enable GPU**: `Runtime` → `Change runtime type` → `Hardware accelerator: GPU`
2. **Run Cell 1**: Sets up the environment (clone repo, install packages)
3. **Run remaining cells** sequentially

## Features:
- Automatic environment setup for Colab
- GPU detection and verification
- Google Drive integration for saving models
- Optimized batch sizes for Colab GPUs


## 🚀 Step 0: Google Colab Setup (RUN THIS FIRST!)


In [ ]:
# ============================================
# GOOGLE COLAB ENVIRONMENT SETUP
# ============================================

import sys
import os

# Detect if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except:
    IN_COLAB = False
    print("✓ Running locally")

if IN_COLAB:
    print("\n" + "="*60)
    print("SETTING UP GOOGLE COLAB ENVIRONMENT")
    print("="*60)
    
    # Check GPU
    import torch
    if torch.cuda.is_available():
        print(f"\n✓ GPU DETECTED: {torch.cuda.get_device_name(0)}")
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  PyTorch Version: {torch.__version__}")
    else:
        print("\n⚠️  WARNING: GPU NOT DETECTED!")
        print("   Go to: Runtime → Change runtime type → Hardware accelerator: GPU")
        print("   Then restart this notebook.")
    
    # Clone repository
    print("\n📦 Cloning repository...")
    # Replace 'YOUR_USERNAME' with your GitHub username
    repo_url = "https://github.com/YOUR_USERNAME/endingengineering.git"
    
    # Alternative: Upload files manually
    print("   Option 1: Clone from GitHub (recommended)")
    print(f"   !git clone {repo_url}")
    print("\n   Option 2: Upload files manually")
    print("   Uncomment the lines below to upload a zip file:")
    print("""
    # from google.colab import files
    # uploaded = files.upload()  # Upload your project.zip
    # !unzip -q project.zip
    """)
    
    # For now, clone (you can modify this)
    # !git clone {repo_url}
    
    # Alternative: Upload files
    print("\n📁 Please choose setup method:")
    print("   A) Clone from GitHub - modify repo_url above and uncomment git clone")
    print("   B) Upload files - uncomment the upload code above")
    print("\n⚠️  After setup, uncomment the appropriate section and rerun this cell")
    
    # Uncomment ONE of these:
    # Method A: GitHub clone
    # !git clone https://github.com/YOUR_USERNAME/endingengineering.git
    # %cd endingengineering
    
    # Method B: Manual upload
    # from google.colab import files
    # import zipfile
    # uploaded = files.upload()
    # for f in uploaded.keys():
    #     if f.endswith('.zip'):
    #         !unzip -q {f}
    #         dir_name = f.replace('.zip', '')
    #         %cd {dir_name}
    
    # Install dependencies
    print("\n📚 Installing dependencies...")
    !pip install -q tqdm seaborn
    
    # Mount Google Drive for saving models
    print("\n💾 Mounting Google Drive (optional - for saving models)...")
    print("   This allows you to save trained models permanently.")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    
    print("\n" + "="*60)
    print("✅ SETUP COMPLETE!")
    print("="*60)
    print("\nYou can now run the remaining cells.")
    print("Note: Make sure to uncomment the setup method above on first run!")
    
else:
    print("Running locally - no Colab setup needed.")


## 1. Setup and Imports


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import os
import sys

# Add project root to path (works for both Colab and local)
try:
    import google.colab
    # Colab paths
    if os.path.exists('/content/endingengineering'):
        sys.path.insert(0, '/content/endingengineering')
    project_root = '/content/endingengineering'
except:
    # Local paths
    project_root = os.path.abspath('.')
    if project_root not in sys.path:
        sys.path.insert(0, project_root)

# Import project modules
import utils.utils as util
import utils.quantization as q
import model.fracbnn_cifar10 as m

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("="*60)
print("ENVIRONMENT CHECK")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"Working directory: {os.getcwd()}")
print("="*60)
